##CHANGELOG
- improved prompts for spymasters
- proper voting for llm teams
- prompting history of the game
- make the function more general (other llms like Claude, Grok etc. can play if play function is implemented; even let humans team play)

In [60]:
#@title import dependencies
!pip install groq
!pip install openai
import requests
import os
from groq import Groq
from tqdm import tqdm
import random
import re
import ast
from openai import OpenAI
from google.colab import userdata
from IPython.display import IFrame, display, HTML

## Utils Functions

In [61]:
prompt_colors = {
    "RED": "\033[1;31m",
    "BLUE": "\033[1;34m",
    "endcolor": "\033[0m",
    "n": "\033[1;33m"
  }

neutral_links = [f"https://github.com/mich1803/Codenames-LLM/blob/main/graphics/NEUTRAL{i}.jpg?raw=true" for i in range(1, 5)]
blue_links = [f"https://github.com/mich1803/Codenames-LLM/blob/main/graphics/BLUE{i}.jpg?raw=true" for i in range(1, 5)]
red_links = [f"https://github.com/mich1803/Codenames-LLM/blob/main/graphics/RED{i}.jpg?raw=true" for i in range(1, 5)]
team_links = {"RED": red_links, "BLUE": blue_links}
killer_link = "https://github.com/mich1803/Codenames-LLM/blob/main/graphics/KILLER.jpg?raw=true"
neutral_link = "https://github.com/mich1803/Codenames-LLM/blob/main/graphics/neutral.jpg?raw=true"

#BOARD GENERATOR
def generate_board(n = 25, lang = "eng", c = 7, k = 1):
  '''
  inputs:
    n (int): number of words
    lang (str): language of words
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''
  blue_cards = c
  red_cards = c + 1
  if (2*c + k + 1) > n:
    raise Exception("ERROR: wrong parameters for board creation")

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/{lang}.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)
  random_words = [word.upper() for word in random_words]

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, blue_cards)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, red_cards)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  colors = [
      "BLUE" if i in blue_indices else
      "RED" if i in red_indices else
      "KILLER" if i in black_index else
      "NEUTRAL"
      for i in range(n)
      ]

  word_color_dict = {random_words[i]: colors[i] for i in range(n)}

  return word_color_dict

#BOARD FORMATTING FOR PROMPT
def board4prompt(word_color_dict, master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()

  return a string version for the prompt for LLMs
  '''

  text = "|"
  if master:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} ({word_color_dict[i]}) " + "|"
      if (idx > 0) and (((idx + 1) % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"
  else:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} " + "|"
      if (idx > 0) and ((idx % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"


  return text

def create_image_dict(word_color_dict, master = False):
  image_dict = {}
  if master:
    for word in word_color_dict:
      if word_color_dict[word] == "NEUTRAL":
        image_dict[word] = random.choice(neutral_links)
      elif word_color_dict[word] == "BLUE":
        image_dict[word] = random.choice(team_links["BLUE"])
      elif word_color_dict[word] == "RED":
        image_dict[word] = random.choice(team_links["RED"])
      elif word_color_dict[word] == "KILLER":
        image_dict[word] = killer_link

  else:
    for word in word_color_dict:
        image_dict[word] = neutral_link

  return image_dict


#STYLISH BOARD FORMATTING FOR PRINT
def board4print(image_dict):
    """
    Inputs:
        image_dict (dict): A dictionary where keys are words, and values are image links.

    Returns:
        str: A formatted HTML string for rendering the game board.
    """
    if not image_dict:
        raise ValueError("The image_dict is empty or invalid.")

    # Start HTML table
    html = """<table style="border-collapse: collapse; margin: 0 auto; width: 60%; text-align: center;">"""

    # Split words into rows (5 columns per row)
    words = list(image_dict.keys())
    rows = [words[i:i + 5] for i in range(0, len(words), 5)]

    for row in rows:
        html += "<tr>"
        for word in row:
            image_link = image_dict[word]
            html += f"""
            <td style="
                font-size: 30px;
                background-image: url('{image_link}');
                background-size: cover;
                background-position: center;
                color: white;
                padding: 10px;
                border-radius: 0px;
                border: 5px solid white; /* Add white border */
                width: 20%;
                height: 100px;
                text-shadow: 3px 3px #171717;
                overflow: hidden; /* Ensures border-radius applies */
            ">
                <b>{word}</b>
            </td>
            """
        html += "</tr>"

    html += "</table>"
    return html




In [62]:
board = generate_board(n = 25, lang = "eng", c = 7, k = 1)
master_image_dict = create_image_dict(board, master = True)
image_dict = create_image_dict(board, master = False)
display(HTML(board4print(image_dict)))
display(HTML('<br><br>'))
display(HTML(board4print(master_image_dict)))


## API unifier

In [78]:
def call_api(system_prompt, prompt, model, json_mode):

  #OPENAI models:
  if model == "gpt-4o-mini": #or other OpenAI models
    API_key = userdata.get('openai')
    client = OpenAI(api_key=API_key)

    #OPENAI normal mode
    if not json_mode:
      chat_completion = client.chat.completions.create(
            model=model,
            messages=[
                      {"role": "system", "content": system_prompt},
                      {"role": "user", "content": prompt}
                      ]
        )
      return chat_completion.choices[0].message.content

    #OPENAI json mode
    if json_mode:
      chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": system_prompt},
                  {"role": "user", "content": prompt}
                  ]
        )
      answer = chat_completion.choices[0].message.content
      #print(f"Raw API Response: {answer}")  # Print the raw response for debugging
      try:
        data = ast.literal_eval(answer)

      except (SyntaxError, ValueError) as e:
        # Catch specific errors and provide more context in the error message
        print(f"Error parsing API response: {e}")
        print(f"Response content: {answer}")
        raise ValueError("Format error: The API response is not a valid Python literal.") from e  # Chain exceptions for better debugging
      return data

  elif model == "grok-beta":
    API_key = userdata.get('xai')
    client = OpenAI(api_key=API_key, base_url="https://api.x.ai/v1")

    #OPENAI normal mode
    if not json_mode:
      chat_completion = client.chat.completions.create(
            model=model,
            messages=[
                      {"role": "system", "content": system_prompt + 'Dont format the code, GIVE ONLY THE RAW DICT'},
                      {"role": "user", "content": prompt}
                      ]
        )
      return chat_completion.choices[0].message.content

    #OPENAI json mode
    if json_mode:
      chat_completion = client.chat.completions.create(
        model=model,
        messages=[
                  {"role": "system", "content": system_prompt},
                  {"role": "user", "content": prompt}
                  ]
        )
      answer = chat_completion.choices[0].message.content
      answer = answer.removeprefix('```json')
      answer = answer.removeprefix('```python')
      answer = answer.removesuffix('```')
      #print(f"Raw API Response: {answer}")  # Print the raw response for debugging
      try:
        data = ast.literal_eval(answer)

      except (SyntaxError, ValueError) as e:
        # Catch specific errors and provide more context in the error message
        print(f"Error parsing API response: {e}")
        print(f"Response content: {answer}")
        raise ValueError("Format error: The API response is not a valid Python literal.") from e  # Chain exceptions for better debugging
      return data

  else:
    raise Exception("ERROR: wrong model name")


## Unify agents

In [75]:
def spymaster(lang, board, team, history, model, cot):
  json_mode = True
  sys_prompt = (
      "We are playing the board game CODENAMES, and your role is the spymaster."+ \
      f"You are part of team {team}, and your job is to help your team guess the words associated with your team's color."+ \
      "You will be given a list of words, each with an associated color: RED for team red words, BLUE for team blue words, "+ \
      "KILLER for killer words, and NEUTRAL for neutral words."+ \
      "Your goal is to provide a ONE-WORD CLUE and a NUMBER. The clue should help your team guess only the words with your team's color."+ \
      "If your team guesses the correct word, you gain 1 point. If your team guesses an opponent's word, they gain 1 point. "+ \
      "If your team guesses the KILLER word, you lose instantly, so be extremely cautious!"+ \
      "Be careful with your clues! Important: DO NOT use any words that are already on the board, as that would immediately make you lose."+ \
      "To ensure you do not accidentally break this rule, follow these steps strictly:"+ \
      "1. Before generating a clue, check the list of words on the board. If the clue you are about to give matches any word on the board, discard it and choose another clue."+ \
      "2. Use synonyms, related concepts, or descriptive ideas to create your clue, ensuring it is not an exact match for any word on the board."+ \
      "3. Double-check your clue against all words on the board before finalizing your response."+ \
      "Your clue should not be too vague or too specific. The clue should help your team make a guess about the words on the board, but it should be safe and avo+ \id leading to mistakes."+ \
      "You should also be mindful of these important factors:"+ \
      "1. Avoid using a word from the board as your clue. If you do, you will lose instantly."+ \
      "2. The clue should not be misleading or overly broad. Think of how it could be interpreted and avoid ambiguities."+ \
      "3. The number you provide should represent how many words on the board are related to your clue."+ \
      "4. The number should be chosen carefully. Do not overestimate how many words fit the clue, and avoid making the number too high, "+ \
      "as it could increase the risk of the guessers making mistakes."+ \
      "5. If your team guesses the KILLER word, you will lose the game."+ \
      "6. Consider the current state of the game: If there are few words left, give more specific clues. If there are many words left, "+ \
      "give broader clues that point to multiple words."+ \
      "7. Always prioritize safety. If a clue could be interpreted in a way that risks your team guessing incorrectly, choose a safer clue."+ \
      "After you choose the clue, double-check if it’s not in the board and strategically helpful for your team.".upper()
)

  if not cot:
    adding_cot = "YOUR RESPONSE SHOULD BE A JSON PYTHON DICT OBJECT WITH TWO KEYS: 'clue' and 'number'. Make sure the clue is a single word, and the number "+ \
      "is an integer between 1 and the number of words left on the board related to that clue."
  else:
    adding_cot = "YOUR RESPONSE SHOULD BE A JSON PYTHON DICT OBJECT WITH THREE KEYS: 'thought', 'clue' and 'number'. Make sure the clue is a single word, and the number "+ \
      "is an integer between 1 and the number of words left on the board related to that clue." +\
      "In the thought (the fisrt thing you are going to say as a 200 words max test) you need to use reasoning to understand which clue and number to say, divide the problem in subproblem like:" +\
      "- which words i cannot say like the ones in the board (or you lose instantly);" +\
      f"- which are the words of your team ({team}) and which word can I say that aligns with them;" +\
      "- which are the words the you are better to avoid make your team think, like the ones of the opponents' team or the killer one." +\
      "at the end double check that your clue is not in the board to avoid losing instantly.".upper() +\
      "BE EXTREMELY SURE THAT THE DICT KEYS ARE RIGHT (thought, clue, number)."

  sys_prompt += adding_cot

  prompt = f"The board is {board4prompt(board, master = True)}, The history of the game is: {history}."

  response = call_api(sys_prompt, prompt, model, json_mode)
  if cot: print(response["thought"]) #test print of chain of thought
  return response["clue"].upper(), response["number"]

def guesser(lang, team, board, clue, n_guessers, i_agent, cards_remaining, idea, k, history, conv, model = "gpt-4o-mini"):
  opp = "BLUE" if team == "RED" else "RED"
  json_mode = True
  sys_prompt = "We are playing the board game CODENAMES, and your role" + \
              f"will be the guesser. \n You're in a team of {n_guessers}" +\
              f"other guessers. Your objective as a team is to guess" +\
              f"the words of your color based on the clue of your spymaster \n" +\
              f"There are: {cards_remaining[team]} words for your team, "+\
              f"{cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
              f"Even if the number said by the spymaster is more than one, let's guess one word at a time. \n" +\
              f"You are in team {team}. You are agent number {i_agent}." +\
              f"Your inital thoughts were: {idea} \n" +\
              f"You are a very good JSON file writer." +\
              f"You need to give in output a PYTHON JSON DICT OBJECT with 3 keys:\n" +\
              f"M: your message to other teammates (be short and coincise); \n" +\
              f"W: a bool that is 1 if you want to listen and speak to the others again, 0 if you feel like you wouldn't add anything to the conversation (*if you all agree please say 0*); \n" +\
              f"V: a dictionary that maps each word in the board to a real number in iterval (-10,10) that represents your confidence in guessing that word (don't be afraid of writing decimal numbers)."

  prompt = f"The board is {board4prompt(board)}. \n" +\
          f"The history is {history}. \n" +\
          f"The clue is: {clue}. \n" +\
          f"The previous conversation is: {conv}."

  response = call_api(sys_prompt, prompt, model, json_mode)
  #print(response["W"])
  return response["M"], bool(response["W"]), response["V"]


def guesser_ideas(lang, team, board, clue, n_guessers, i_agent, cards_remaining, k, history,  model = "gpt-4o-mini"):
  opp = "BLUE" if team == "RED" else "RED"
  json_mode = False
  sys_prompt = "We are playing the board game CODENAMES, and your role" + \
              f"will be the guesser. \n You're in a team of {n_guessers}" +\
              f"other guessers. Your objective as a team is to guess" +\
              f"the words of your color based on the clue of your spymaster \n" +\
              f"There are: {cards_remaining[team]} words for your team, "+\
              f"{cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
              f"Even if the number said by the spymaster is more than one, let's guess one word at a time. \n" +\
              f"You are in team {team}. You are agent number {i_agent}." +\
              "Try to say a max of 200 characters."

  prompt = f"The board is {board4prompt(board)}. \n" +\
          f"The history is {history}. \n" +\
          f"The clue is: {clue}. \n" +\
          "Share your (short and coincise) initial thoughts with your other fellow teammates."

  response = call_api(sys_prompt, prompt, model, json_mode)
  return response



## Vote system

In [65]:
def vote_system(votes):
  results = {}
  for vote in votes:
    for word, points in vote.items():
      if word in results:
        results[word] += points
      else:
        results[word] = points
  return max(results, key=results.get)

## Turno

In [73]:
def play_turn(lang, team, board, cards_remaining, k, n_guessers, history, image_dict, master_image_dict, model, cot, verbose = False, masterverbose = False):
  opp = "BLUE" if team == "RED" else "RED"
  rc = cards_remaining
  b = board

  if verbose:
    print("The actual board is: ")
    if masterverbose:
      display(HTML(board4print(master_image_dict)))
    else:
      display(HTML(board4print(image_dict)))
    print(f"it is {prompt_colors[team]}{team}{prompt_colors['endcolor']} team turn: \n")

  if model == "human":
    clue = input(f"Insert the clue for the {team} team:").upper()
    number = input(f"Insert the number of words for the {team} team:")
    spymaster_history = f"Team {team} spymaster said: {clue} ({number})."
    guesser_history = []

    if clue in b:
      return {"endgame": True, "win": opp, "reason": "spymaster said a word in the board"}

    for i in range(int(number)):
      guess = input(f"Insert the guess for the {team} team:").upper()

      try:
          x = b[guess]
      except:
          if verbose: print(f"{prompt_colors[team]}{team}{prompt_colors['endcolor']} team guess wasn't on the board. \n \n")
          guesser_history.append(f"Team {team} said: {guess}. The word was not in board")
          return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}


      image_dict[guess] = master_image_dict[guess]
      del master_image_dict[guess]
      if x == "KILLER":
          if verbose: print(f"The killer word have been selected, the game ends. \n \n ")

          return {"endgame": True, "win": opp, "reason": "guessed killer word"}

      elif x == team:
          rc[team] -= 1
          if verbose: print(f'A {prompt_colors[team]}{team} word{prompt_colors["endcolor"]} have been selected ({team} cards remaining = {rc[team]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {team}.")
          if rc[team] == 0:
              if verbose: print(f"The {prompt_colors[team]}{team} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": team, "reason": "cards finished"}

      elif x == opp:
          rc[opp] -= 1
          if verbose: print(f'A {prompt_colors[opp]}{opp} word{prompt_colors["endcolor"]} have been selected ({opp} cards remaining = {rc[opp]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {opp}.")
          if rc[opp] == 0:
              if verbose: print(f"The {prompt_colors[opp]}{opp} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": opp, "reason": "cards finished"}
          break

      else:
          guesser_history.append(f"Team {team} said: {guess}. The word was neutral.")
          if verbose: print(f"A neutral word have been selected. \n \n")
          del b[guess]
          break

    #return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}


  else:
    clue, number = spymaster(lang, board, team, history, model, cot)
    spymaster_history = f"Team {team} spymaster said: {clue} ({number})."
    guesser_history = []
    if verbose: print(f"The {prompt_colors[team]}{team}{prompt_colors['endcolor']} spymaster's clue is: {clue} ({number}). \n \n")

    if clue in b:
      return {"endgame": True, "win": opp, "reason": "spymaster said a word in the board"}


    for i in range(number):
      ideas = []
      votes = [{} for _ in range(n_guessers)]
      want_to_talk = [True for _ in range(n_guessers)]
      conv = []
      for j in range(n_guessers):
        ideas.append(guesser_ideas(lang, team, board, clue, n_guessers, j, cards_remaining, k, history,  model))
        if verbose: print(f"Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} guesser {j} thinks: {ideas[j]}.")
        conv.append(f"Team {team} guesser {j} said: {ideas[j]}.")
      speaktoomuch = 0
      while (any(want_to_talk)) and speaktoomuch < 2:
        if verbose: print(f"Do you want to talk? {want_to_talk}")
        for j in range(n_guessers):
          if want_to_talk[j]:
            mess, want_to_talk[j], votes[j] = guesser(lang, team, board, clue, n_guessers, j, cards_remaining, ideas[j], k, history, conv, model)
            if verbose: print(f"Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} guesser {j} said: {mess}.")
            conv.append(f"Team {team} guesser {j} said: {mess}.")
        speaktoomuch += 1
      guess = vote_system(votes)
      if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} voted: {guess}. \n \n")


      try:
          x = b[guess]
      except:
          if verbose: print(f"{prompt_colors[team]}{team}{prompt_colors['endcolor']} team guess wasn't on the board. \n \n")
          guesser_history.append(f"Team {team} said: {guess}. The word was not in board")
          return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}

      image_dict[guess] = master_image_dict[guess]
      del master_image_dict[guess]
      if x == "KILLER":
          if verbose: print(f"The killer word have been selected, the game ends. \n \n ")
          return {"endgame": True, "win": opp, "reason": "guessed killer word"}

      elif x == team:
          rc[team] -= 1
          if verbose: print(f'A {prompt_colors[team]}{team} word{prompt_colors["endcolor"]} have been selected ({team} cards remaining = {rc[team]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {team}.")
          if rc[team] == 0:
              if verbose: print(f"The {prompt_colors[team]}{team} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": team, "reason": "cards finished"}

      elif x == opp:
          rc[opp] -= 1
          if verbose: print(f'A {prompt_colors[opp]}{opp} word{prompt_colors["endcolor"]} have been selected ({opp} cards remaining = {rc[opp]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {opp}.")
          if rc[opp] == 0:
              if verbose: print(f"The {prompt_colors[opp]}{opp} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": opp, "reason": "cards finished"}
          break

      else:
          guesser_history.append(f"Team {team} said: {guess}. The word was neutral.")
          if verbose: print(f"A neutral word have been selected. \n \n")
          del b[guess]
          break

  return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}

In [67]:
# prompt: write a function to send an email to the human spymaster with the html board

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(recipient_email, html_board):
    """Sends an email with the HTML board to the specified recipient.

    Args:
        recipient_email: The email address of the recipient.
        html_board: The HTML content of the board.
        sender_email: Your email address (default: your_email@gmail.com).
                      **Replace with your actual email address.**
        sender_password: Your email password (default: your_password).
                         **Replace with your actual password or an app password.**
    """
    sender_email = userdata.get('codenames-mail')
    sender_password = userdata.get('codenames-mail-app')
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = "Codenames Board"

    msg.attach(MIMEText(html_board, 'html'))

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(sender_email, sender_password)
            smtp.send_message(msg)
        print("Master board sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

## Gioco

In [71]:
#@title play game function
def play_game(lang = "eng", n_cards = 25, coloured_cards = 7, k_cards = 1, verbose = False, red_model = "gpt-4o-mini", red_cot = True, blue_model = "gpt-4o-mini", blue_cot = True, red_agents = 3, blue_agents = 3, masterverbose = False):
  r = 1
  turn = "RED"
  board = generate_board(n = n_cards, lang = lang, c = coloured_cards, k = k_cards)
  image_dict = create_image_dict(board, master = False)
  master_image_dict = create_image_dict(board, master = True)
  initial_board = master_image_dict.copy()
  cards_remaining = {"RED": coloured_cards + 1, "BLUE": coloured_cards}
  history = []

  if red_model == "human":
    mail = input("Insert the mail of the red spymaster:")
    send_email(mail, board4print(master_image_dict))
  if blue_model == "human":
    mail = input("Insert the mail of the blue spymaster:")
    send_email(mail, board4print(master_image_dict))
  #input("stop")

  if verbose:
      intro = prompt_colors["n"] + "GAME PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"language = {lang}, \n" + \
      f"number of cards = {n_cards}, \n" + \
      f"number of killer cards = {k_cards}, \n" + \
      "\n" + \
      prompt_colors["RED"] + "RED TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      "Red team starts first, " + \
      f"number of red cards = {cards_remaining['RED']}, \n" + \
      f"red model = {red_model}, \n" + \
      f"number of red agents = {red_agents}, \n" + \
      "\n" + \
      prompt_colors["BLUE"] + "BLUE TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"number of blue cards = {cards_remaining['BLUE']}, \n" + \
      f"blue model = {blue_model}, \n" + \
      f"number of blue agents = {blue_agents}, \n"

      print("-----------------------------------")
      print(intro)
      print("-----------------------------------")

  while True:
    result = play_turn(lang = lang,
                       team = turn,
                       board = board,
                       cards_remaining = cards_remaining,
                       k = k_cards,
                       n_guessers = red_agents if turn == "RED" else blue_agents,
                       history = history,
                       image_dict = image_dict,
                       master_image_dict = master_image_dict,
                       model = red_model if turn == "RED" else blue_model,
                       cot = red_cot if turn == "RED" else blue_cot,
                       verbose = verbose,
                       masterverbose = masterverbose)
    if result["endgame"]:
      if verbose:
        print(result)
        display(HTML(board4print(initial_board))) # Changed this line
      return (result["win"], result["reason"], r)
    else:
      cards_remaining = result["rc"]
      board = result["b"]
      r += 1
      turn = "BLUE" if turn == "RED" else "RED"
      history.append(result["spyh"])
      history += result["teamh"]

In [69]:
play_game(verbose = True, masterverbose = True)

-----------------------------------
GAME PARAMETERS:
language = eng, 
number of cards = 25, 
number of killer cards = 1, 

RED TEAM PARAMETERS:
Red team starts first, number of red cards = 8, 
red model = gpt-4o-mini, 
number of red agents = 3, 

BLUE TEAM PARAMETERS:
number of blue cards = 7, 
blue model = gpt-4o-mini, 
number of blue agents = 3, 

-----------------------------------
The actual board is: 


it is RED team turn: 

To come up with a suitable clue for my team, I must first identify the words that are safe to use without leading to an instant loss. The red words available to me are: BEAT, CENTAUR, ROSE, SCALE, UNDERTAKER, LIMOUSINE, and PIRATE. I need to avoid using any words on the board: QUEEN, SHOE, YARD, MAMMOTH, KEY, TABLET, BEIJING, WATER, THUMB, HELICOPTER, SHADOW, and NIGHT. The words I want to focus on must clearly relate to my red team words without being too general and potentially misleading. I want a clue that connects multiple red words but keeps my team's guessing focused and safe from the KILLER or BLUE words. The word 'NIGHT' could relate to themes of darkness, mystery, or things typically associated with night-time. However, it's also a RED word, so it's not an option. Based on my analysis, the word 'PIRATE' stands out as one that connects to adventure and has a historical connotation. I can use 'TREASURE' as a clue which relates to 'PIRATE' and may connect 

it is BLUE team turn: 

I need to carefully choose a clue that helps my team guess BLUE words without triggering any RED or KILLER words. The BLUE words available are MAMMOTH, TABLET, BEIJING, PASTE, WATER, and THUMB. The RED words to avoid include BEAT, CENTAUR, ROSE, SCALE, UNDERTAKER, and LIMOUSINE, while YARD is the KILLER word. Since MAMMOTH could relate to a historical context, I can consider a clue related to ancient creatures or items that might suggest multiple BLUE words. One possibility includes thinking about TABLET and BEIJING relating to history or ancient civilization. However, I need to be cautious with my clue not to overlap with the KILLER and to avoid vague associations with RED. I believe the term 'ANCIENT' could work, and for safety, I can point to 2 BLUE words. After checking again, 'ANCIENT' does not appear on the board, making it safe.
The BLUE spymaster's clue is: ANCIENT (2). 
 

Team BLUE guesser 0 thinks: For the clue "ANCIENT," I'm thinking of words that re

it is RED team turn: 

Looking at the current board, I need to identify which words are associated with my team, RED: BEAT, ROSE, SCALE, UNDERTAKER, LIMOUSINE, and NIGHT. The opponents' words to avoid are TABLET, BEIJING, PASTE, WATER, and THUMB. Furthermore, I must steer clear of the KILLER word YARD to avoid an instant loss. I want to create a clue that helps my team associate with multiple RED words but can be misconstrued. A strong choice would be a clue related to something connected with our words without using any words directly from the board. Since NIGHT is involved, I might consider concepts like darkness or evening. Given there are several RED words that could link to a theme of nighttime, I could use 'DARK' as a clue, since it relates to NIGHT and can also imply a mood that connects with possibly BEAT (like a dark beat) or even a dramatic theme that could be related to UNDERTAKER or ROSE (which could symbolize darkness in a poetic sense). The number I will use is 3, as I be

it is BLUE team turn: 

I need to help my team guess the BLUE words: TABLET, BEIJING, PASTE, WATER, and THUMB. However, I must avoid any words already on the board: QUEEN, SHOE, BEAT, YARD, BARK, ROSE, SCALE, NOTE, UNDERTAKER, LIMOUSINE, PIPE, STATE, AIR, and HELICOPTER. Since my focus is on the BLUE words, I should think of clues indirectly related to them but not too common, which could lead my team to guess incorrectly. It's also crucial to avoid any references that might accidentally connect to RED or the KILLER word (YARD). To create a clue that could possibly relate to multiple BLUE words, I might consider something linked to technology or something that signifies movement or fluidity, which can relate to both WATER and PROCESSING (like tapping on a TABLET). After deciding on a clue, I need to double check and ensure it's a term that isn’t on the board. I believe 'DEVICE' could work but could mislead to RED or neutral words. Instead, I'll go with 'FLUID', relating to the liquid a

...

it is BLUE team turn: 

To select a clue for Team BLUE, I need to analyze the words on the board and find connections to our team's words without risking an instant loss. The BLUE words on the board are TABLET, BEIJING, PASTE, and THUMB. I need to avoid using any of these words in my clue. Additionally, I should steer clear of the KILLER word, YARD, and RED words like BEAT, ROSE, SCALE, and UNDERTAKER. Since our team has four potential words related to technology or communication, I can use a clue like 'TECH' to hint towards TABLET and possibly PASTE. I'm considering the number '2' as it’s a safe guess since it covers TABLET and PASTE, while avoiding the more ambiguous connections that might lead to confused guesses. After carefully checking, 'TECH' is not on the board, making it a safe option. Finally, I will relay this clue with the number 2, which reflects the association without overstating the connections. This will help guide my teammates effectively without making errors.
The BL

it is RED team turn: 

First, we need to identify which words I cannot use as clues because they are on the board: SHOE, BEAT, YARD, BARK, ROSE, SCALE, NOTE, UNDERTAKER, BEIJING, PASTE, STATE, AIR, and THUMB. The words associated with Team RED that I want to guide my team towards are BEAT, ROSE, SCALE, and UNDERTAKER. However, we must also be cautious about the KILLER word, which is YARD. It would be dangerous if my clue could lead to that or to any BLUE words. To maximize the chance of safe guesses, I want a clue that connects with at least two RED words. A fitting clue could be 'FUNERAL', which relates to UNDERTAKER and may indirectly link to BEAT (as in music played). Choosing this clue targets two RED words without touching any neutral, blue, or killer words. Thus, I’ll suggest the clue 'FUNERAL' with a number of 2 to keep the focus clear and safe.
The RED spymaster's clue is: FUNERAL (2). 
 

Team RED guesser 0 thinks: For the clue FUNERAL, I’m thinking about related words like UN

it is BLUE team turn: 

I'm analyzing the current state of the board and history. I see that our team BLUE has the words BEIJING, PASTE, and THUMB associated with it. We need a clue that will lead us to guess these words without hitting the KILLER word (YARD), which is dangerous, or the RED words (BEAT, SCALE). Since we want to make a connection that is safe and specific enough, I’ll look for a term associated with BEIJING, PASTE, and THUMB. The idea of 'writing' could be tied to 'PASTE' and 'THUMB' since thumbs are commonly used for gripping or typing. However, 'writing' may be too broad and lead us to potential RED words. Rather than 'writing,' I think 'ASIA' could work well to direct us toward BEIJING and potentially towards PASTE for its online context. I’m considering the number I should associate with my clue. Since 'ASIA' strongly relates to BEIJING, I can confidently say there’s at least one word related to the clue. I’ll provide a one-word clue connected to BEIJING. This shoul

it is RED team turn: 

Looking at the current board, the words available are SHOE (NEUTRAL), BEAT (RED), YARD (KILLER), BARK (NEUTRAL), SCALE (RED), NOTE (NEUTRAL), PASTE (BLUE), STATE (NEUTRAL), AIR (NEUTRAL), and THUMB (BLUE). My goal is to come up with a one-word clue that will help my team guess the two RED words (BEAT and SCALE) while avoiding the KILLER word (YARD) and the BLUE words (PASTE and THUMB). I need to be cautious about the terms I can use for my clue to ensure none are present in the board. The word 'GAME' seems safe, relating to activities that can have beats (such as music or sports) and scales (like scoring). The number '2' is appropriate since it relates to both RED words. Since no words from the board match or could be confused with 'GAME,' it is a safe clue choice. Checking again, I confirm that neither RED word is present in the clue, and it does not lead to any risk of guessing incorrectly.
The RED spymaster's clue is: GAME (2). 
 

Team RED guesser 0 thinks: F

('BLUE', 'guessed killer word', 9)

In [77]:
play_game(red_model = "human", verbose = True, masterverbose = False)

Insert the mail of the red spymaster:frmarrocco03@gmail.com
Master board sent successfully!
-----------------------------------
GAME PARAMETERS:
language = eng, 
number of cards = 25, 
number of killer cards = 1, 

RED TEAM PARAMETERS:
Red team starts first, number of red cards = 8, 
red model = human, 
number of red agents = 3, 

BLUE TEAM PARAMETERS:
number of blue cards = 7, 
blue model = gpt-4o-mini, 
number of blue agents = 3, 

-----------------------------------
The actual board is: 


it is RED team turn: 

Insert the clue for the RED team:WATER
Insert the number of words for the RED team:2
Insert the guess for the RED team:WHALE
A RED word have been selected (RED cards remaining = 7). 
 

Insert the guess for the RED team:GRASS
A BLUE word have been selected (BLUE cards remaining = 6). 
 

The actual board is: 


it is BLUE team turn: 

I need to focus on providing a clue that relates to the BLUE words on the board without overlapping with any existing words, particularly avoiding BANK, which is the killer word. The BLUE words are ROME, CHEST, GERMANY, CRASH, MOUTH, and PIE. I will consider descriptive or related concepts that can lead my team to these words. My challenge is to convey a clue that encompasses multiple BLUE words but maintains clarity. A good approach might be to think about concepts that relate to things commonly associated with these words. For instance, both GERMANY and CHEST bring to mind concepts of exploration. ROME could relate to history or travel, while CRASH might connect to accidents, potentially invoked during discussions about risk during explorations. However, since PIE is a food item, I want to keep the clue broad, yet safe, that does not lead them to any RED words. Thus, I will consider using the clue ‘TRAVEL’, which can encompass both historical and modern contex

it is RED team turn: 

Insert the clue for the RED team:DEMOCRACY
Insert the number of words for the RED team:2
Insert the guess for the RED team:AMERICA
A RED word have been selected (RED cards remaining = 5). 
 

Insert the guess for the RED team:GERMANY
A BLUE word have been selected (BLUE cards remaining = 4). 
 

The actual board is: 


it is BLUE team turn: 

I need to provide a clue that relates to the BLUE words while avoiding the RED, KILLER, and NEUTRAL words. The words associated with team BLUE are CHEST, CRASH, MOUTH, and PIE. I have to discard any clue that directly matches any of the words on the board to avoid an instant loss. The KILLER word is BANK, which I need to be especially cautious of, as guessing that would end the game immediately for my team. Considering the BLUE words, they relate to themes of money (CHEST), impact (CRASH), body (MOUTH), and food (PIE). I could go with something related to achieving success or gaining resources, like 'TREASURE,' which connects with CHEST and could hint at others. CHEST is a direct correlation for treasure, while CRASH could be interpreted as crashing into a treasure, MOUTH could suggest consumption, and PIE suggests a slice of resources. However, I want to be specific and avoid multiple interpretations. I decide on the clue 'TREASURE' with a number of '3' to high

('RED', 'guessed killer word', 4)